In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from konlpy.tag import Okt

In [3]:
pd.set_option('display.unicode.east_asian_width',True) #행열 정렬

In [4]:
df = pd.read_csv('./crawling_data/raw_65548_2021-06-18.csv', engine='python')
print(df.head())
print(df.info())

                         title  \
0    황성주 박사의 생식과 건강   
1  아름다운 청년, 대니 서의 집   
2           여왕 패션 종이인형   
3       더 포스터 북 by 이미소   
4                    가죽 소품   

                                                                                summary  \
0  생식을 통한 식생활 개선으로 성인병을 치료하는 방법서. 육류 등의 포화지방이나 콜레...   
1  환경 운동가 대니 서가 공개하는 환경 친화적인 집 꾸미기. 이 책은 교외에 위치한 ...      
2  다양한 업적을 가진 세계의 여왕 패션을 직접 입혀보고, 아름답고 다양한 드레스와 왕...    
3  직관적인 제목처럼 권마다 포스터 작품 10점으로 가득 채워진 도서이다. 그러나 일반...     
4  가죽공예의 기본 테크닉에서부터 명함 지갑, 동전 지갑, 물림쇠 지갑, 파우치, 안경...      

           category  
0  Home_Cook_Beauty  
1  Home_Cook_Beauty  
2  Home_Cook_Beauty  
3  Home_Cook_Beauty  
4  Home_Cook_Beauty  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65548 entries, 0 to 65547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     65548 non-null  object
 1   summary   65516 non-null  object
 2   category  65548 non-null  object
dtypes: object(3)
memory u

In [5]:
df= df.dropna(axis=0)
print(df.head())
print(df.info())

                         title  \
0    황성주 박사의 생식과 건강   
1  아름다운 청년, 대니 서의 집   
2           여왕 패션 종이인형   
3       더 포스터 북 by 이미소   
4                    가죽 소품   

                                                                                summary  \
0  생식을 통한 식생활 개선으로 성인병을 치료하는 방법서. 육류 등의 포화지방이나 콜레...   
1  환경 운동가 대니 서가 공개하는 환경 친화적인 집 꾸미기. 이 책은 교외에 위치한 ...      
2  다양한 업적을 가진 세계의 여왕 패션을 직접 입혀보고, 아름답고 다양한 드레스와 왕...    
3  직관적인 제목처럼 권마다 포스터 작품 10점으로 가득 채워진 도서이다. 그러나 일반...     
4  가죽공예의 기본 테크닉에서부터 명함 지갑, 동전 지갑, 물림쇠 지갑, 파우치, 안경...      

           category  
0  Home_Cook_Beauty  
1  Home_Cook_Beauty  
2  Home_Cook_Beauty  
3  Home_Cook_Beauty  
4  Home_Cook_Beauty  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 65516 entries, 0 to 65547
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     65516 non-null  object
 1   summary   65516 non-null  object
 2   category  65516 non-null  object
dtypes: object(3)
memory u

In [6]:
col_dup = df['summary'].duplicated()
print(col_dup)
sum_dup = df.summary.duplicated().sum()
print(sum_dup)

0        False
1        False
2        False
3        False
4        False
         ...  
65543     True
65544    False
65545    False
65546     True
65547    False
Name: summary, Length: 65516, dtype: bool
7252


In [7]:
df = df.drop_duplicates(subset=['summary'])
sum_dup = df.summary.duplicated().sum()
print(sum_dup) #중복된 개수의 합

0


In [8]:
#인덱스를 처음부터 다시 붙여주는것
df.reset_index(drop=True, inplace=True )
#drop=True를 해야 기존 인덱스를 없애줌

In [9]:
X = df['summary']
Y= df['category']

In [10]:
len(X)
print(X[1250:1350])


1250    우리나라에도 미식문화가 자리를 잡아감에 따라 보다 전문적이고 참신한 레스토랑 가이드...
1251    골판지를 이용한 환경구성 작품을 제작하는 동안 쌓인 노하우를 실은 책이다. 꽃과 동...
1252    개인이나 의사 한두 명이 아닌 국내 최초 소아전문 한의원 ‘함소아’에서 13년 동안...
1253    최근 몇 년간 얼굴이 작고 갸름한 동안미녀, 세련된 훈남으로 변신한 연예인들의 양악...
1254    이나영, 수애, 이병헌, 한가인, 김효진, 유지태, 임창정, 김남길, 엄태웅 등 톱...
                                            ...                                      
1345    신혼 주부, 요리에 자신이 없는 젊은 엄마, 바쁜 맞벌이 부부, 그리고 독신자를 위...
1346    간단 아침식사에서부터 국수 한접시, 한그릇 밥요리, 별미반찬과 술안주까지 한그릇에 ...
1347    크고 기다란 바게트를 머리에 얹고 마른 옥수수를 양 어깨에 늘어뜨린 여자의 흉상이나...
1348    전통 음식은 한방 의학을 기초로 한 음식의 조리와 음양오행설의 반영이 특징이라는 개...
1349    오직 초보자를 위해 하나부터 열까지 세심하게 배려해 만든 키트다. 그중 <나의 첫 ...
Name: summary, Length: 100, dtype: object


In [11]:
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label)

['Art_Culture' 'Economic' 'Economic_Management' 'Health_Hobby_Leisure'
 'Home_Cook_Beauty' 'Novel_Poem' 'Reference_Book' 'Religion_Mysticism']


In [12]:
with open('./data/book_category_encoder.pickle', 'wb') as f:
  pickle.dump(encoder, f)

print(labeled_Y)

[4 4 4 ... 1 1 1]


In [13]:
onehot_Y = to_categorical(labeled_Y)
print(onehot_Y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [14]:
stopwords = pd.read_csv('./stopwords.csv')

In [15]:
okt = Okt()
print(type(X))

<class 'pandas.core.series.Series'>


In [16]:
okt_X = okt.morphs(X[0]) # 첫번째 title만 형태소 분리를 해서 okt_X에 넣어줌
print(X[0])
print(okt_X)

생식을 통한 식생활 개선으로 성인병을 치료하는 방법서. 육류 등의 포화지방이나 콜레스테롤, 설탕을 많이함유하고 있는 요즘 음식은 건강을 해친다. 일반식에 비해 에너지 효율이 5,6배 높은 생식를 먹고 병이 나 은 임상결과를 다양한 사례와 함께 실었다.
['생식', '을', '통한', '식생활', '개선', '으로', '성인병', '을', '치료', '하는', '방법', '서', '.', '육류', '등', '의', '포화지방', '이나', '콜레스테롤', ',', '설탕', '을', '많이', '함유', '하고', '있는', '요즘', '음식', '은', '건강', '을', '해친다', '.', '일반', '식', '에', '비해', '에너지', '효율', '이', '5', ',', '6', '배', '높은', '생', '식', '를', '먹고', '병', '이', '나', '은', '임', '상', '결과', '를', '다양한', '사례', '와', '함께', '실었다', '.']


In [17]:
for i in range(len(X)):
  X[i] = okt.morphs(X[i])
print(X)

0        [생식, 을, 통한, 식생활, 개선, 으로, 성인병, 을, 치료, 하는, 방법, 서...
1        [환경, 운동, 가, 대니, 서가, 공개, 하는, 환경, 친, 화, 적, 인, 집,...  
2        [다양한, 업적, 을, 가진, 세계, 의, 여왕, 패션, 을, 직접, 입혀, 보고,...
3        [직관, 적, 인, 제목, 처럼, 권, 마다, 포스터, 작품, 10, 점, 으로, ...   
4        [가죽공예, 의, 기본, 테크닉, 에서부터, 명함, 지갑, ,, 동전, 지갑, ,,...
                                          ...                                   
58259    [전, 세계, 가, 코로나, 19, 바이러스, 로, 신음, 하던, 2020년, 5월...    
58260    [IT, Cookbook, 한빛, 교재, 시리즈, 362, 권, ., 자동, 제어공...         
58261    [--, 한국, 과학, 문화재단, ', 우수, 과학, 인증, 도서, ',, 「, 시...    
58262    [인간, 을, 비롯, 한, 모든, 생명체, 는, 미생물, 에서, 비롯, 되었다, ....
58263    [모든, 물질, 의, 기본, 인, 원소, 를, 깔끔하게, 보고, 직관, 적, 으로,...
Name: summary, Length: 58264, dtype: object


In [18]:
for i in range(len(X)):
  result = []
  for j in range(len(X[i])):
    if len(X[i][j]) > 1:
      if X[i][j] not in list(stopwords['stopword']):
        result.append(X[i][j])
  X[i] = ' '.join(result)
  
print(X)

0        생식 통한 식생활 개선 성인병 치료 방법 육류 포화지방 이나 콜레스테롤 설탕 많이 ...
1        환경 운동 대니 서가 공개 환경 꾸미기 교외 위치 부모님 평범한 주택 환경 이고 세... 
2        다양한 업적 가진 세계 여왕 패션 직접 입혀 아름답고 다양한 드레스 왕관 품위 여왕...
3        직관 제목 마다 포스터 작품 10 가득 채워진 도서 일반 종이 평범하게 인쇄 작품 ...   
4        가죽공예 기본 테크닉 에서부터 명함 지갑 동전 지갑 물림 지갑 파우치 안경집 소품 ...
                                                ...                                        
58259    세계 코로나 19 바이러스 신음 하던 2020년 5월 모처럼 세계 곳곳 이목 집중 ...       
58260    IT Cookbook 한빛 교재 시리즈 362 자동 제어공학 처음 배우는 공학 계열...           
58261    -- 한국 과학 문화재단 우수 과학 인증 도서 ', 시사저널 올해 2003년 선정 ...        
58262    인간 비롯 모든 생명체 미생물 비롯 되었다 인간 병원 미생물 역사 관계 핏빛 물들어...
58263    모든 물질 기본 원소 깔끔하게 직관 이해 있도록 비주 발견 118 가지 원소 성질 ...    
Name: summary, Length: 58264, dtype: object


In [19]:
token = Tokenizer()
token.fit_on_texts(X) # 어떤형태소를 무슨 숫자로 만들지 지정
tokened_X = token.texts_to_sequences(X) # 해당 형태소를 라벨링 된 숫자로 변경
print(tokened_X[0])

[5422, 883, 2202, 1029, 6851, 194, 8, 4580, 25850, 46, 6639, 2988, 289, 6778, 78, 90, 59543, 348, 1752, 588, 672, 314, 1108, 251, 13, 89, 543]


In [23]:
with open('./data/books_token.pickle', 'wb') as f:
  pickle.dump(token, f)

ERROR! Session/line number was not unique in database. History logging moved to new session 646


In [24]:
wordsize = len(token.word_index) +1 #나중에 0을 포함하려고
print(wordsize)

131128


In [25]:
max = 0
for i in range(len(tokened_X)):
  if max < len(tokened_X[i]):
    max = len(tokened_X[i])

print(max)

970


In [26]:
X_pad = pad_sequences(tokened_X, max) #토큰의 개수가 16개가 되게끔 알아서 앞쪽에 0을 붙여준다
print(X_pad[:10])

[[   0    0    0 ...   13   89  543]
 [   0    0    0 ...    2  390  665]
 [   0    0    0 ...  258 6082  502]
 ...
 [   0    0    0 ... 3001 1750 1503]
 [   0    0    0 ...  139  302 1329]
 [   0    0    0 ... 9237  684 1842]]


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad, onehot_Y, test_size=0.15
)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(49524, 970)
(8740, 970)
(49524, 8)
(8740, 8)


In [28]:
xy = X_train, X_test, Y_train, Y_test
np.save('./data/books_data_max_{}_size_{}'.format(max,wordsize), xy)


C:\Users\tyler\anaconda3\envs\AI_exam\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
ERROR! Session/line number was not unique in database. History logging moved to new session 647
